<a href="https://colab.research.google.com/github/amedeotolace/Amedeo_Thesis_2/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow
!pip install tensorflow_datasets
!pip install tensorflow tensorflow_decision_forests
!pip install datapackage
!pip install pandas
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.3 MB/s eta 0:00:00
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10746 sha256=d58a0c0e560e6de613085ffda2297f986cdc8c0f8e1c29989df6705f0538de1d
  Stored in directory: /root/.

In [17]:
import datapackage
import pandas as pd

data_url = 'https://datahub.io/machine-learning/kddcup99/datapackage.json'

# to load Data Package into storage
package = datapackage.Package(data_url)

# to load only tabular data
resources = package.resources
for resource in resources:
    if resource.tabular:
        dataset_df = pd.read_csv(resource.descriptor['path'])
        print (dataset_df)

        duration protocol_type service flag  src_bytes  dst_bytes  land  \
0              0           tcp    http   SF        181       5450     0   
1              0           tcp    http   SF        239        486     0   
2              0           tcp    http   SF        235       1337     0   
3              0           tcp    http   SF        219       1337     0   
4              0           tcp    http   SF        217       2032     0   
...          ...           ...     ...  ...        ...        ...   ...   
494015         0           tcp    http   SF        310       1881     0   
494016         0           tcp    http   SF        282       2286     0   
494017         0           tcp    http   SF        203       1200     0   
494018         0           tcp    http   SF        291       1200     0   
494019         0           tcp    http   SF        219       1234     0   

        wrong_fragment  urgent  hot  ...  dst_host_srv_count  \
0                    0       0    0

In [37]:
#Split del dataset in train & test

import numpy as np

def split_dataset(dataset, test_ratio=0.30):
  """Divide un pandas dataframe, in questo caso suddiviamo il dataset caricato in 2"""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]


train_ds_pd, test_ds_pd = split_dataset(dataset_df)

#Lunghezza dei dataset di test e di train
"""print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))
"""
print(test_ds_pd.T)

                             3       7       15      18      22      25      \
duration                          0       0       0       0       0       0   
protocol_type                   tcp     tcp     tcp     tcp     tcp     tcp   
service                        http    http    http    http    http    http   
flag                             SF      SF      SF      SF      SF      SF   
src_bytes                       219     159     260     233     241     248   
dst_bytes                      1337    4087    1837     255     259    2129   
land                              0       0       0       0       0       0   
wrong_fragment                    0       0       0       0       0       0   
urgent                            0       0       0       0       0       0   
hot                               0       0       0       0       0       0   
num_failed_logins                 0       0       0       0       0       0   
logged_in                         1       1       1 

In [35]:
#Random Forest

import tensorflow_decision_forests as tfdf
import pandas as pd

#Convertiamo da Pandas dataframe ad un dataset Tensorflow
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label='protocol_type')
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label='protocol_type')

# Random Forest model.
model = tfdf.keras.RandomForestModel()
model.fit(train_ds)

# Summary of the model structure.
model.summary()

# Evaluate the model.
model.evaluate(test_ds)

# Export the model to a SavedModel.
model.save("project/model")

Use /tmp/tmpxbvhicrv as temporary training directory
Reading training dataset...
Training dataset read in 0:00:12.846533. Found 345706 examples.
Training model...
Model trained in 0:01:16.334497
Compiling model...
Model compiled.
Model: "random_forest_model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (41):
	count
	diff_srv_rate
	dst_bytes
	dst_host_count
	dst_host_diff_srv_rate
	dst_host_rerror_rate
	dst_host_same_src_port_rate
	dst_host_same_srv_rate
	dst_host_serror_rate
	dst_host_srv_count
	dst_host_srv_diff_host_rate
	dst_host_srv_rerror_rate
	dst_host_srv_serror_rate
	duration
	flag
	hot
	is_guest_login
	is_host_login
	label
	land
	lnum_access_files
	lnum_co